In [ ]:
%load_ext autoreload
%autoreload 2
import sys
import os

PROJECT_ROOT=os.path.join(os.path.dirname(os.path.abspath(os.pardir)))
sys.path.append(PROJECT_ROOT)

In [ ]:
import torch
import torchvision
import numpy as np


from PIL import Image
from datetime import date

from object_detection.transform import Invertor
from object_detection import mnist_augmentation
from object_detection import mnist_inference
from object_detection import mnist_evaluation


In [ ]:
CONFIG = {"batch_size":200}
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
SAVE_DPATH = os.path.join(PROJECT_ROOT,'results','mnist_augmentations')
os.makedirs(SAVE_DPATH, exist_ok=True)

In [ ]:
# загружаем тестовую выборку
test_data = torchvision.datasets.MNIST(
   os.path.join(PROJECT_ROOT, "mnist_content"), train=False, transform=Invertor(), download=True
)

# Создаём лоядеры данных.
# так как модель ожидает данные в определённой форме
test_dataloader=torch.utils.data.DataLoader(
    dataset=test_data, 
    batch_size=CONFIG["batch_size"],
    shuffle=False
)

Добавляем в тестовые данные альбументации 

In [ ]:
transforms = mnist_augmentation.AlbuAugmentation()

test_data_with_augmentation = []
for data in test_data:
    img, label = data
    img = np.array(img)
    # adding albumentations
    transformed_img = transforms(img)

    # Transform array to type PIL.Image.Image 
    transformed_img = Image.fromarray(transformed_img)

    # Collect data examples
    test_data_with_augmentation.append([transformed_img,label])

In [ ]:
checkpoint_dpath = os.path.join(PROJECT_ROOT, "checkpoints", "mnist_checkpoints")
model_fpath = os.path.join(checkpoint_dpath, "best.pth")

infer = mnist_inference.Inference.from_file(model_fpath, device= DEVICE)

In [ ]:
mnist_evaluator= mnist_evaluation.MnistEvaluator(infer,test_data_with_augmentation)

Получим предсказания для тестовых данных с альбументациями

In [ ]:
_SAVE_PRED= True

predictions = mnist_evaluator.evaluate()

if _SAVE_PRED:
    predictions.to_csv(os.path.join(SAVE_DPATH,f'pred_with_aug_{date.today()}.csv'),index_label='id')


Получим метрики для данных с альбументациями

In [ ]:
_SAVE_METRICS=True

metrics =mnist_evaluator.classification_report()

print(f'Classification report with augmentations')
print(metrics)

if _SAVE_METRICS:
    metrics.to_csv(os.path.join(SAVE_DPATH,f'classification_report_with_aug_{date.today()}.csv'),index_label='label')

